# More COBRA models

Now that we have the basics down, let's try some other analysis

## Flux Variability Analysis.

Recall that FVA holds an objective function constant, and then optimizes for the minimum and maximum value of each flux that can maintain that maximum. Think of this as exploring the flat edges of a high-dimensional polygon.

In [ ]:
from cobra.io import read_sbml_model

In [ ]:
model = read_sbml_model('data/iJO1366.xml.gz')

In [ ]:
from cobra.flux_analysis import flux_variability_analysis

In [ ]:
fvaresults = flux_variability_analysis(model)
fvaresults
subset = fvaresults.sort_values(by='maximum').head(15) # This will just take the first 15 entries, sorted by maximum

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"]=15,4
plt.bar(x=subset.index, height=subset.maximum, bottom = subset.minimum) 
   
    # check out https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.bar.html

In [ ]:
model.optimize()
model.summary(fva=0.95)

## Simulating gene deletions

We can take advantage of the gene to reaction linkage to simulate the effect of gene deletions. This is going to be similar to manually setting reaction fluxes to 0, but accounts for more realistic situations:

1. When a gene catalyzes more than one reaction
2. When more than one gene product can catalyze the same reaction

Let's try deleting phosphofructokinase!

First, let's find the associated genes.

In [ ]:
model.reactions.query("PFK")

In [ ]:
[print(reaction.name, reaction) for reaction in model.reactions.query("PFK")]

In [ ]:
model.reactions.PFK.genes

We see that two genes, b1723 and b3916, encode phosphofructokinase. Let's try single mutants as well as double mutants.

In [ ]:
print('complete model: ', model.optimize())
with model: # This format prevents overwriting the original model
    model.genes.b1723.knock_out()
    print('pfkA knocked out: ', model.optimize()) 
with model:
    model.genes.b3916.knock_out()
    print('pfkB knocked out: ', model.optimize()) 
with model:
    model.genes.b1723.knock_out()
    model.genes.b3916.knock_out()
    print('pfkB and pfkB knocked out: ', model.optimize()) 

Unsurprisingly, single deletions don't slow cell growth, because another isoform can (in theory) pick up the slack. But a double deletion isn't so bad in these conditions, either! How could this be the case?

In [ ]:
newmodel = model.copy()
newmodel.genes.b1723.knock_out()
newmodel.genes.b3916.knock_out()
nopfk = newmodel.optimize()

In [ ]:
import escher
escher.Builder(map_name='iJO1366.Central metabolism',
                   model = newmodel,
                   reaction_data=dict(nopfk.fluxes))

Hmm, it seems that F6PA is picking up the slack from the loss of PFK. We can predict that deleting the genes involved in this reaction would be fatal. Let's test.

First, can we check for differences programatically?

In [ ]:
diff = newmodel.optimize().fluxes - model.optimize().fluxes

In [ ]:
diff.sort_values()

Yes, F6PA and DHAPT seem to increase in flux just as much as PFK and FBA drop.

In [ ]:
newmodel.reactions.F6PA.genes

There are two genes involved with F6PA, let's delete both of them and see if our quadruple mutant survives.

In [ ]:
with newmodel:
    newmodel.genes.b0825.knock_out()
    print('F6PA_1 knocked out: ', newmodel.optimize()) 
with newmodel:
    newmodel.genes.b0825.knock_out()
    newmodel.genes.b3946.knock_out()
    print('F6PA_1 and F6PA_2 knocked out: ', newmodel.optimize()) 

What have we learned from this? We've demonstrated that central metabolism in E. coli can be very robust to gene deletions! And by using mass balance, we can see that textbook components of glycolysis can be removed with very little change in growth rate. With some loops, you could fairly easily write a script to test the effect of every pairwise, triple or quadruple mutant!

## Finding all essential genes

We've seen how we could go through cycles of hypothesis generation and prediction, and we can foresee how a brute force approach could work. Let's take a brute force approach to identify all essential genes.

Fortunately, cobrapy has a function to do this for us.

In [ ]:
from cobra.flux_analysis import find_essential_genes
essential = find_essential_genes(model)
[print(gene.name) for gene in essential];

# Pathway engineering

By now I hope I've demonstrated how we can use genome-scale metabolic models to predict phenotypes. 
Can we now use this approach to tie new pathways into a cell, and estimate whether if we can produce reasonable yields? Can we predict changes in growth rate?

We'll create some reactions and metabolites first, and then place them into the model. We'll need to import some functions from cobra to do this.

In [ ]:
from cobra import Reaction, Metabolite
model = read_sbml_model('data/iJO1366.xml.gz')

We'll create a reaction called alchemy

In [ ]:
new_reaction = Reaction('alchemy')

It will turn phosphate into gold, so we'll need to define gold as a metabolite.

In [ ]:
gold = Metabolite(id='gold_c', compartment='c', name='GOLD')

Now that we have a metabolite, we can construct a reaction around it. We'll add metabolites to the reaction object. We'll have an ATP consumed, generating gold and an ATP. This will permantly remove a phosphate from the system, but return ADP as a metabolite.

In [ ]:
new_reaction.add_metabolites({model.metabolites.atp_c: -1, gold: 1, model.metabolites.adp_c: 1})

We'll add the new reaction to the model. Note that the add_reactions method requires a list input, even if it is just one item.

In [ ]:
model.add_reactions([new_reaction])

If we were to run the simulation now, it would fail to produce gold, because there is no demand being placed upon it. Rather than an export reaction, we'll say that gold is leaving the system. We can define a new boundary condition as follows.

In [ ]:
model.add_boundary(model.metabolites.gold_c, type='demand')

We're still going to use the biomass objective function, so there will be no "incentive" to produce gold. We'll force the system by introducing a positive, lower boundary.

In [ ]:
model.reactions.alchemy.bounds = (200, 1000)

In [ ]:
model.reactions.alchemy

In [ ]:
model.objective.expression

In [ ]:
opt = model.optimize()

In [ ]:
print("Biomass flux:", opt.fluxes.BIOMASS_Ec_iJO1366_core_53p95M)
print("Gold flux:", opt.fluxes.alchemy)

We can see that this puts a heavy burden on cell growth rates. We can predict that the cell would be starved of phosphate, and by using some of the analytical tools described so far (reduced costs, shadow price, FVA), you could imagine how we could go about determining bottlenecks.

We could also do a deletion series and see if there are any E. coli proteins that are now detrimental for growth under these conditions. Maybe their deletion would have slowed growth in non-productive cells, but now perhaps there pathways that act as a drain on the system.

This wraps up the basics, but going forward, there are lots of approaches to formalize strain engineering by considering growth rates!